In [ ]:
#import torch
#print(torch.cuda.is_available())   # should return True


In [1]:
# @title Default title text
# Fetch dataset
!git clone https://github.com/amir9ume/urdu_ghazals_rekhta.git

# Check files
!ls urdu_ghazals_rekhta



Cloning into 'urdu_ghazals_rekhta'...
remote: Enumerating objects: 112, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 112 (delta 7), reused 6 (delta 6), pack-reused 103 (from 1)
Receiving objects: 100% (112/112), 2.03 MiB | 23.66 MiB/s, done.
Resolving deltas: 100% (51/51), done.
dataset  LICENSE  README.md  rekhta_parser.ipynb  sample_dataset


In [2]:
!unzip urdu_ghazals_rekhta/dataset/dataset.zip -d urdu_ghazals_rekhta/dataset/

Archive:  urdu_ghazals_rekhta/dataset/dataset.zip
   creating: urdu_ghazals_rekhta/dataset/dataset/
   creating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/
   creating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/silsile-tod-gayaa-vo-sabhii-jaate-jaate-ahmad-faraz-ghazals  
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/kyaa-aise-kam-sukhan-se-koii-guftuguu-kare-ahmad-faraz-ghazals  
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/havaa-ke-zor-se-pindaar-e-baam-o-dar-bhii-gayaa-ahmad-faraz-ghazals  
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/avval-avval-kii-dostii-hai-abhii-ahmad-faraz-ghazals  
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/saaqiyaa-ek-nazar-jaam-se-pahle-pahle-ahmad-faraz-ghazals  
  inflating: urdu_ghazals_rekhta/dataset/dataset/ahmad-faraz/ur/saamne-us-ke-kabhii-us-kii-sataaish-nahiin-kii-ahmad-faraz-ghazals  
  inflat

In [ ]:
import os

# list all files inside the dataset folder
os.listdir("urdu_ghazals_rekhta/dataset/dataset")


['akbar-allahabadi',
 'habib-jalib',
 'nida-fazli',
 'bahadur-shah-zafar',
 'allama-iqbal',
 'sahir-ludhianvi',
 'ameer-khusrau',
 'jaun-eliya',
 'parveen-shakir',
 'gulzar',
 'noon-meem-rashid',
 '.DS_Store',
 'faiz-ahmad-faiz',
 'ahmad-faraz',
 'jaan-nisar-akhtar',
 'dagh-dehlvi',
 'meer-anees',
 'mohsin-naqvi',
 'kaifi-azmi',
 'meer-taqi-meer',
 'naseer-turabi',
 'altaf-hussain-hali',
 'nazm-tabatabai',
 'naji-shakir',
 'javed-akhtar',
 'mirza-ghalib',
 'waseem-barelvi',
 'fahmida-riaz',
 'wali-mohammad-wali',
 'firaq-gorakhpuri',
 'jigar-moradabadi']

In [ ]:
!file urdu_ghazals_rekhta/dataset/dataset/altaf-hussain-hali/en/aage-badhe-na-qissa-e-ishq-e-butaan-se-ham-altaf-hussain-hali-ghazals-3

urdu_ghazals_rekhta/dataset/dataset/altaf-hussain-hali/en/aage-badhe-na-qissa-e-ishq-e-butaan-se-ham-altaf-hussain-hali-ghazals-3: cannot open `urdu_ghazals_rekhta/dataset/dataset/altaf-hussain-hali/en/aage-badhe-na-qissa-e-ishq-e-butaan-se-ham-altaf-hussain-hali-ghazals-3' (No such file or directory)


# What I have done currently now:
1- first i have enable GPU by going into noteBook settings in the hardware appearance.

2- Second I feteched the data set from github repo and see the file/folder available init by using the 'ls' command.

3- Third I have done is unzipping the dataset.zip available here:
'urdu_ghazals_rekhta/dataset'.

4- By writing this code:

```
import os

# list all files inside the dataset folder
os.listdir("urdu_ghazals_rekhta/dataset/dataset")


# This is formatted as code
```
I have seen all the folders available init.
Also there in each of the folder there were three subfolders:

**1-** "en"

**2-** "ur"

**3-** "hi"

now in each if these folders there are text files in which data is written. Here, the data which is required by us is in the folder
**'en'** and **'ur'** but the fact is that they are not cleaned.

 let's move forward and try to clean the data.

---



In [3]:
import re
import unicodedata
import pandas as pd
from pathlib import Path
import csv
import os

# --- Configuration ---
INPUT_DIR = "urdu_ghazals_rekhta/dataset/dataset"
OUTPUT_FILE = "urdu_roman_pairs.csv"
CHUNK_SIZE = 10000

# --- Cleaning functions ---
URDU_DIACRITICS = re.compile("[\u0610-\u061A\u064B-\u065F\u06D6-\u06ED]")
TATWEEL = "\u0640"
URDU_ALLOWED = re.compile(r"[^\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF\uFB50-\uFDFF\uFE70-\uFEFF\s\.\?\!،؛:\-]")

def clean_urdu(text):
    if not text or not isinstance(text, str):
        return ""

    text = unicodedata.normalize("NFC", text)
    text = text.replace(TATWEEL, "")
    text = URDU_DIACRITICS.sub("", text)
    text = text.replace("\u064A", "\u06CC")
    text = text.replace("\u0649", "\u06CC")
    for alef in ["\u0622", "\u0623", "\u0625"]:
        text = text.replace(alef, "\u0627")
    text = URDU_ALLOWED.sub("", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text
def clean_roman(text):
    if not text or not isinstance(text, str):
        return ""

    text = text.lower()

    # SMART DASH HANDLING:

    # Case 1: Internal word dashes (like "sta-e-tahzb-e-a") - REMOVE
    # These are usually single letters separated by dashes within a word
    text = re.sub(r'(\b\w)-(\w\b)', r'\1\2', text)

    # Case 2: Word separator dashes (like "shayari-ka-mausam") - REPLACE WITH SPACE
    # These separate actual words
    text = text.replace('-', ' ')

    # Remove other unwanted characters
    text = re.sub(r"[^a-zA-Z\s\.\?\!']", "", text)

    # Clean up multiple spaces
    text = re.sub(r"\s+", " ", text).strip()

    return text

def is_valid_pair(urdu_text, roman_text):
    if not urdu_text or not roman_text:
        return False
    if len(urdu_text) < 3 or len(roman_text) < 3:
        return False
    urdu_chars = re.findall(r'[\u0600-\u06FF]', urdu_text)
    if len(urdu_chars) / max(1, len(urdu_text)) < 0.5:
        return False
    alpha_chars = re.findall(r'[a-zA-Z]', roman_text)
    if len(alpha_chars) / max(1, len(roman_text)) < 0.5:
        return False
    return True

# --- Simple function to detect text files ---
def is_text_file(file_path):
    """Check if a file is likely a text file"""
    try:
        # Check if it's a file (not directory)
        if not file_path.is_file():
            return False

        # Check file size (skip very large files)
        if file_path.stat().st_size > 10 * 1024 * 1024:  # 10MB limit
            return False

        # Check if it has a text-like extension or no extension
        if file_path.suffix.lower() in ['.txt', '.text', '.utf8', '.utf-8', '']:
            return True

        # Try to read first few bytes to check if it's text
        with open(file_path, 'rb') as f:
            sample = f.read(1024)
            # Very basic text detection: check if mostly printable bytes
            printable_count = sum(32 <= byte <= 126 or byte in [9, 10, 13] for byte in sample)
            if printable_count / len(sample) > 0.7:  # 70% printable
                return True

        return False

    except:
        return False

# --- Process all files ---
def process_all_data(input_dir, output_file):
    input_path = Path(input_dir)

    if not input_path.exists():
        print(f"Error: Input directory '{input_dir}' does not exist!")
        return 0

    # Initialize CSV file
    with open(output_file, 'w', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['urdu', 'roman_urdu'])

    total_pairs = 0
    chunk = []
    processed_files = 0

    # Iterate through all author directories
    for author_dir in input_path.iterdir():
        if not author_dir.is_dir():
            continue

        ur_dir = author_dir / "ur"
        en_dir = author_dir / "en"

        # Skip if either directory doesn't exist
        if not ur_dir.exists() or not en_dir.exists():
            print(f"Skipping {author_dir.name}: missing ur or en folder")
            continue

        # Get ALL files (not just .txt)
        en_files = [f for f in en_dir.iterdir() if f.is_file()]
        ur_files = [f for f in ur_dir.iterdir() if f.is_file()]

        print(f"Processing {author_dir.name}: {len(en_files)} en files, {len(ur_files)} ur files")

        if not en_files or not ur_files:
            print(f"  No files found in {author_dir.name}")
            continue

        # Create a mapping of base filenames (without extensions)
        en_file_map = {f.stem: f for f in en_files}  # stem = filename without extension
        ur_file_map = {f.stem: f for f in ur_files}

        # Find common files by base name
        common_files = set(en_file_map.keys()) & set(ur_file_map.keys())

        print(f"  Found {len(common_files)} matching file pairs")

        # Process each matching file pair
        for base_name in common_files:
            en_file = en_file_map[base_name]
            ur_file = ur_file_map[base_name]

            try:
                # Read both files with UTF-8 encoding
                with open(ur_file, 'r', encoding='utf-8') as f:
                    ur_lines = f.read().splitlines()
                with open(en_file, 'r', encoding='utf-8') as f:
                    en_lines = f.read().splitlines()

                print(f"  Processing {base_name}: {len(ur_lines)} Urdu lines, {len(en_lines)} Roman lines")

                # Process each line pair
                file_pairs = 0
                for u, e in zip(ur_lines, en_lines):
                    cu = clean_urdu(u)
                    ce = clean_roman(e)

                    if is_valid_pair(cu, ce):
                        chunk.append((cu, ce))
                        total_pairs += 1
                        file_pairs += 1

                        # Write in chunks to manage memory
                        if len(chunk) >= CHUNK_SIZE:
                            with open(output_file, 'a', encoding='utf-8', newline='') as f:
                                writer = csv.writer(f)
                                writer.writerows(chunk)
                            chunk = []
                            print(f"    Processed {total_pairs} pairs total...")

                print(f"    Added {file_pairs} valid pairs from {base_name}")
                processed_files += 1

            except UnicodeDecodeError:
                # Try with different encoding if UTF-8 fails
                try:
                    with open(ur_file, 'r', encoding='latin-1') as f:
                        ur_lines = f.read().splitlines()
                    with open(en_file, 'r', encoding='latin-1') as f:
                        en_lines = f.read().splitlines()

                    print(f"  Processing {base_name} with latin-1 encoding: {len(ur_lines)} Urdu lines, {len(en_lines)} Roman lines")

                    file_pairs = 0
                    for u, e in zip(ur_lines, en_lines):
                        cu = clean_urdu(u)
                        ce = clean_roman(e)

                        if is_valid_pair(cu, ce):
                            chunk.append((cu, ce))
                            total_pairs += 1
                            file_pairs += 1

                            if len(chunk) >= CHUNK_SIZE:
                                with open(output_file, 'a', encoding='utf-8', newline='') as f:
                                    writer = csv.writer(f)
                                    writer.writerows(chunk)
                                chunk = []
                                print(f"    Processed {total_pairs} pairs total...")

                    print(f"    Added {file_pairs} valid pairs from {base_name}")
                    processed_files += 1

                except Exception as e:
                    print(f"Error processing {ur_file} with latin-1: {e}")
                    continue

            except Exception as e:
                print(f"Error processing {ur_file}: {e}")
                continue

    # Write any remaining pairs
    if chunk:
        with open(output_file, 'a', encoding='utf-8', newline='') as f:
            writer = csv.writer(f)
            writer.writerows(chunk)

    print(f"Processed {processed_files} file pairs")
    return total_pairs

# --- Debug directory structure ---
def debug_directory_structure(input_dir):
    input_path = Path(input_dir)
    print("Debugging directory structure...")

    if not input_path.exists():
        print(f"ERROR: Input directory '{input_dir}' does not exist!")
        return False

    author_dirs = [d for d in input_path.iterdir() if d.is_dir()]
    print(f"Found {len(author_dirs)} author directories")

    for author_dir in author_dirs[:3]:  # Check first 3 authors
        print(f"\nChecking {author_dir.name}:")

        # Check what subdirectories exist
        subdirs = [d for d in author_dir.iterdir() if d.is_dir()]
        print(f"  Subdirectories: {[d.name for d in subdirs]}")

        # Check en directory
        en_dir = author_dir / "en"
        if en_dir.exists():
            en_files = list(en_dir.iterdir())
            print(f"  en/ files: {len(en_files)} files")
            for f in en_files[:5]:  # Show first 5 files
                if f.is_file():
                    print(f"    {f.name} (size: {f.stat().st_size} bytes, ext: {f.suffix})")
        else:
            print("  en/ directory does not exist!")

        # Check ur directory
        ur_dir = author_dir / "ur"
        if ur_dir.exists():
            ur_files = list(ur_dir.iterdir())
            print(f"  ur/ files: {len(ur_files)} files")
            for f in ur_files[:5]:
                if f.is_file():
                    print(f"    {f.name} (size: {f.stat().st_size} bytes, ext: {f.suffix})")
        else:
            print("  ur/ directory does not exist!")

    return True

# --- Test the cleaning function ---
def test_roman_cleaning():
    """Test function to verify Roman Urdu cleaning"""
    test_cases = [
        "sta-e-tahzb-e-a",
        "hello-world-test",
        "normal text",
        "another-test-case",
        "simple",
        "a-b-c-d-e-f"
    ]

    print("Testing Roman Urdu cleaning function:")
    print("-" * 50)
    for test in test_cases:
        cleaned = clean_roman(test)
        print(f"Input:  '{test}'")
        print(f"Output: '{cleaned}'")
        print("-" * 30)

# --- Main execution ---
if __name__ == "__main__":
    # First test the cleaning function
    test_roman_cleaning()

    print("\n" + "=" * 60)
    print("Processing all Urdu-Roman Urdu pairs...")
    print(f"Input directory: {INPUT_DIR}")
    print(f"Output file: {OUTPUT_FILE}")
    print("=" * 60)

    # First debug the directory structure
    debug_directory_structure(INPUT_DIR)

    print("\n" + "=" * 60)
    print("Starting processing...")
    print("=" * 60)

    total_pairs = process_all_data(INPUT_DIR, OUTPUT_FILE)

    print(f"\nProcessing complete! Found {total_pairs} valid pairs.")
    print(f"Data saved to {OUTPUT_FILE}")

    if os.path.exists(OUTPUT_FILE) and total_pairs > 0:
        # Show file info
        file_size = os.path.getsize(OUTPUT_FILE) / (1024 * 1024)
        print(f"File size: {file_size:.2f} MB")

        # Load sample
        try:
            df = pd.read_csv(OUTPUT_FILE, nrows=10)
            print("\nFirst 5 rows:")
            print(df.head())

            # Check if there are any remaining dashes in the Roman Urdu column
            dash_count = df['roman_urdu'].str.contains('-').sum()
            print(f"\nRows with remaining dashes: {dash_count}")
            if dash_count > 0:
                print("Rows with dashes:")
                print(df[df['roman_urdu'].str.contains('-', na=False)]['roman_urdu'].head())

        except Exception as e:
            print(f"Could not read CSV file: {e}")
    else:
        print("Error: No valid pairs found.")

Testing Roman Urdu cleaning function:
--------------------------------------------------
Input:  'sta-e-tahzb-e-a'
Output: 'sta e tahzb ea'
------------------------------
Input:  'hello-world-test'
Output: 'hello world test'
------------------------------
Input:  'normal text'
Output: 'normal text'
------------------------------
Input:  'another-test-case'
Output: 'another test case'
------------------------------
Input:  'simple'
Output: 'simple'
------------------------------
Input:  'a-b-c-d-e-f'
Output: 'ab cd ef'
------------------------------

Processing all Urdu-Roman Urdu pairs...
Input directory: urdu_ghazals_rekhta/dataset/dataset
Output file: urdu_roman_pairs.csv
Debugging directory structure...
Found 30 author directories

Checking nazm-tabatabai:
  Subdirectories: ['en', 'ur', 'hi']
  en/ files: 26 files
    kyaa-kaarvaan-e-hastii-guzraa-ravaa-ravii-men-nazm-tabaa-tabaaii-ghazals (size: 2013 bytes, ext: )
    tanhaa-nahiin-huun-gar-dil-e-diivaana-saath-hai-nazm-tabaa-tabaa

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your cleaned data
df = pd.read_csv("urdu_roman_pairs.csv")
print(f"Total pairs: {len(df)}")

# Split into 50% train, 25% validation, 25% test
train_df, temp_df = train_test_split(df, test_size=0.5, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Verify the splits
print(f"\nAfter splitting:")
print(f"Training set: {len(train_df)} pairs ({len(train_df)/len(df)*100:.1f}%)")
print(f"Validation set: {len(val_df)} pairs ({len(val_df)/len(df)*100:.1f}%)")
print(f"Test set: {len(test_df)} pairs ({len(test_df)/len(df)*100:.1f}%)")

# Save the splits
train_df.to_csv("train.csv", index=False)
val_df.to_csv("validation.csv", index=False)
test_df.to_csv("test.csv", index=False)

print("\nSplits saved to:")
print("- train.csv")
print("- validation.csv")
print("- test.csv")

Total pairs: 21003

After splitting:
Training set: 10501 pairs (50.0%)
Validation set: 5251 pairs (25.0%)
Test set: 5251 pairs (25.0%)

Splits saved to:
- train.csv
- validation.csv
- test.csv


In [5]:
import pandas as pd
import numpy as np
from collections import Counter
import os
import json
import torch
from torch.nn.utils.rnn import pad_sequence

# Load your splits
train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("validation.csv")
test_df = pd.read_csv("test.csv")

print("Dataset sizes:")
print(f"Train: {len(train_df)}, Validation: {len(val_df)}, Test: {len(test_df)}")

# STEP 1: CHARACTER-LEVEL TOKENIZATION
def build_char_vocab(text_series, min_frequency=1):
    """Build character vocabulary from text series"""
    char_counter = Counter()

    print(f"Processing {len(text_series)} texts for vocabulary...")

    for i, text in enumerate(text_series):
        if pd.notna(text) and isinstance(text, str):
            for char in str(text):
                char_counter[char] += 1

        if (i + 1) % 1000 == 0:
            print(f"  Processed {i + 1} texts...")

    print(f"Found {len(char_counter)} unique characters before filtering")

    # Create vocabulary with special tokens
    special_tokens = ['<pad>', '<unk>', '<sos>', '<eos>']
    chars = special_tokens + [
        char for char, count in char_counter.items()
        if count >= min_frequency and char not in special_tokens
    ]

    char_to_idx = {char: idx for idx, char in enumerate(chars)}
    idx_to_char = {idx: char for idx, char in enumerate(chars)}

    vocab_info = {
        'char_to_idx': char_to_idx,
        'idx_to_char': idx_to_char,
        'vocab_size': len(chars),
        'character_counts': dict(char_counter.most_common(50))
    }

    return vocab_info

def text_to_char_sequences(text_series, vocab, max_length=100):
    """Convert text to character-level integer sequences using PyTorch"""
    sequences = []
    char_to_idx = vocab['char_to_idx']

    print(f"Converting {len(text_series)} texts to sequences...")

    for i, text in enumerate(text_series):
        if pd.notna(text) and isinstance(text, str):
            char_sequence = [char_to_idx['<sos>']]

            for char in str(text):
                char_sequence.append(char_to_idx.get(char, char_to_idx['<unk>']))

            char_sequence.append(char_to_idx['<eos>'])
            sequences.append(torch.tensor(char_sequence, dtype=torch.long))
        else:
            sequences.append(torch.tensor([char_to_idx['<sos>'], char_to_idx['<eos>']], dtype=torch.long))

        if (i + 1) % 1000 == 0:
            print(f"  Converted {i + 1} texts...")

    # Pad sequences using PyTorch's pad_sequence
    padded_sequences = pad_sequence(
        sequences,
        batch_first=True,
        padding_value=char_to_idx['<pad>']
    )

    # Truncate if longer than max_length
    if padded_sequences.size(1) > max_length:
        padded_sequences = padded_sequences[:, :max_length]

    return padded_sequences.numpy()  # Convert to numpy for saving

# STEP 2: Build Character Vocabularies
print("\nBuilding character vocabularies...")

# Build Urdu character vocabulary
print("Building Urdu character vocabulary...")
urdu_char_vocab = build_char_vocab(train_df['urdu'], min_frequency=1)

# Build Roman Urdu character vocabulary
print("Building Roman Urdu character vocabulary...")
roman_char_vocab = build_char_vocab(train_df['roman_urdu'], min_frequency=1)

print(f"\nUrdu character vocabulary size: {urdu_char_vocab['vocab_size']}")
print(f"Roman Urdu character vocabulary size: {roman_char_vocab['vocab_size']}")

# STEP 3: Convert all data to character sequences
max_length = 100

print("\nConverting texts to character sequences...")

# Convert all data
train_urdu = text_to_char_sequences(train_df['urdu'], urdu_char_vocab, max_length)
train_roman = text_to_char_sequences(train_df['roman_urdu'], roman_char_vocab, max_length)
val_urdu = text_to_char_sequences(val_df['urdu'], urdu_char_vocab, max_length)
val_roman = text_to_char_sequences(val_df['roman_urdu'], roman_char_vocab, max_length)
test_urdu = text_to_char_sequences(test_df['urdu'], urdu_char_vocab, max_length)
test_roman = text_to_char_sequences(test_df['roman_urdu'], roman_char_vocab, max_length)

print(f"\nCharacter Sequence Shapes:")
print(f"Train Urdu: {train_urdu.shape}, Train Roman: {train_roman.shape}")
print(f"Val Urdu: {val_urdu.shape}, Val Roman: {val_roman.shape}")
print(f"Test Urdu: {test_urdu.shape}, Test Roman: {test_roman.shape}")

# STEP 4: Save character-level processed data
char_folder = "Processed_Char_level_PyTorch"
os.makedirs(char_folder, exist_ok=True)

def save_char_vocab(vocab, filepath):
    """Save character vocabulary with proper JSON serialization"""
    vocab_to_save = {
        'char_to_idx': vocab['char_to_idx'],
        'idx_to_char': {str(k): v for k, v in vocab['idx_to_char'].items()},
        'vocab_size': vocab['vocab_size']
    }
    with open(filepath, 'w', encoding='utf-8') as f:
        json.dump(vocab_to_save, f, ensure_ascii=False, indent=2)

save_char_vocab(urdu_char_vocab, f"{char_folder}/urdu_char_vocab.json")
save_char_vocab(roman_char_vocab, f"{char_folder}/roman_char_vocab.json")

# Save sequences as numpy arrays
np.save(f"{char_folder}/train_urdu.npy", train_urdu)
np.save(f"{char_folder}/train_roman.npy", train_roman)
np.save(f"{char_folder}/val_urdu.npy", val_urdu)
np.save(f"{char_folder}/val_roman.npy", val_roman)
np.save(f"{char_folder}/test_urdu.npy", test_urdu)
np.save(f"{char_folder}/test_roman.npy", test_roman)

# STEP 5: Create PyTorch Dataset class for training
class UrduRomanDataset(torch.utils.data.Dataset):
    """PyTorch Dataset for Urdu to Roman Urdu translation"""

    def __init__(self, urdu_sequences, roman_sequences):
        self.urdu_sequences = torch.tensor(urdu_sequences, dtype=torch.long)
        self.roman_sequences = torch.tensor(roman_sequences, dtype=torch.long)

    def __len__(self):
        return len(self.urdu_sequences)

    def __getitem__(self, idx):
        urdu_seq = self.urdu_sequences[idx]
        roman_seq = self.roman_sequences[idx]

        # For teacher forcing: decoder input is roman_seq[:-1], target is roman_seq[1:]
        decoder_input = roman_seq[:-1]
        target = roman_seq[1:]

        return {
            'encoder_input': urdu_seq,
            'decoder_input': decoder_input,
            'target': target
        }

def create_data_loaders(batch_size=32):
    """Create PyTorch DataLoaders for training, validation, and test"""

    # Load the processed data
    train_urdu = np.load(f"{char_folder}/train_urdu.npy")
    train_roman = np.load(f"{char_folder}/train_roman.npy")
    val_urdu = np.load(f"{char_folder}/val_urdu.npy")
    val_roman = np.load(f"{char_folder}/val_roman.npy")
    test_urdu = np.load(f"{char_folder}/test_urdu.npy")
    test_roman = np.load(f"{char_folder}/test_roman.npy")

    # Create datasets
    train_dataset = UrduRomanDataset(train_urdu, train_roman)
    val_dataset = UrduRomanDataset(val_urdu, val_roman)
    test_dataset = UrduRomanDataset(test_urdu, test_roman)

    # Create data loaders
    train_loader = torch.utils.data.DataLoader(
        train_dataset, batch_size=batch_size, shuffle=True
    )
    val_loader = torch.utils.data.DataLoader(
        val_dataset, batch_size=batch_size, shuffle=False
    )
    test_loader = torch.utils.data.DataLoader(
        test_dataset, batch_size=batch_size, shuffle=False
    )

    return train_loader, val_loader, test_loader

# STEP 6: Load function for main training code
def load_char_vocab(filepath):
    """Load character vocabulary and convert keys back to integers"""
    with open(filepath, 'r', encoding='utf-8') as f:
        vocab_data = json.load(f)

    # Convert idx_to_char keys back to integers
    vocab_data['idx_to_char'] = {int(k): v for k, v in vocab_data['idx_to_char'].items()}

    return vocab_data

# STEP 7: Test encoding/decoding to verify everything works
def char_sequence_to_text(sequence, vocab):
    """Convert character sequence back to text"""
    idx_to_char = vocab['idx_to_char']
    pad_idx = vocab['char_to_idx']['<pad>']
    sos_idx = vocab['char_to_idx']['<sos>']
    eos_idx = vocab['char_to_idx']['<eos>']

    text = ''
    for idx in sequence:
        if idx in [pad_idx, sos_idx, eos_idx]:
            continue
        char = idx_to_char.get(str(idx), idx_to_char.get(idx, '<unk>'))
        text += char
    return text

print(f"\n{'=' * 60}")
print("VERIFICATION TEST")
print(f"{'=' * 60}")

if len(train_df) > 0:
    # Test with current vocab
    test_sentence = train_df['urdu'].iloc[0]
    encoded_sequence = train_urdu[0]
    decoded_text = char_sequence_to_text(encoded_sequence, urdu_char_vocab)

    print(f"Original Urdu: '{test_sentence}'")
    print(f"Decoded Urdu: '{decoded_text}'")
    print(f"Match: {test_sentence == decoded_text}")

    # Test with loaded vocab
    urdu_vocab_loaded = load_char_vocab(f"{char_folder}/urdu_char_vocab.json")
    decoded_text_loaded = char_sequence_to_text(encoded_sequence, urdu_vocab_loaded)
    print(f"Decoded with loaded vocab: '{decoded_text_loaded}'")
    print(f"Match with loaded vocab: {test_sentence == decoded_text_loaded}")

print(f"\nPROCESSING COMPLETE!")
print(f"Files saved in: {char_folder}")
print(f"Urdu character vocab size: {urdu_char_vocab['vocab_size']}")
print(f"Roman character vocab size: {roman_char_vocab['vocab_size']}")

# STEP 8: Final information for PyTorch model training
print(f"\nFOR PYTORCH MODEL TRAINING:")
print(f"Urdu vocab size for embedding layer: {urdu_char_vocab['vocab_size']}")
print(f"Roman vocab size for output layer: {roman_char_vocab['vocab_size']}")
print(f"Max sequence length: {max_length}")

print(f"\nUsage in PyTorch training code:")
print(f"""
# Load the processed data:
urdu_vocab = load_char_vocab('{char_folder}/urdu_char_vocab.json')
roman_vocab = load_char_vocab('{char_folder}/roman_char_vocab.json')

# Create data loaders:
train_loader, val_loader, test_loader = create_data_loaders(batch_size=32)

# Example usage in training loop:
for batch in train_loader:
    encoder_input = batch['encoder_input']  # Shape: (batch_size, seq_len)
    decoder_input = batch['decoder_input']  # Shape: (batch_size, seq_len-1)
    target = batch['target']                # Shape: (batch_size, seq_len-1)

    # Your model forward pass here
    # predictions = model(encoder_input, decoder_input)
    # loss = criterion(predictions, target)
""")

# STEP 9: Utility function for creating masks (useful for PyTorch models)
def create_mask(sequences, pad_idx):
    """Create mask for sequences (1 for valid tokens, 0 for padding)"""
    return (sequences != pad_idx).long()

# Example of how to use masks in PyTorch
print(f"\nMASKING EXAMPLE FOR PYTORCH:")
print("""
# In your model or training loop:
pad_idx = roman_vocab['char_to_idx']['<pad>']

# Create masks
encoder_mask = create_mask(encoder_input, pad_idx)
decoder_mask = create_mask(decoder_input, pad_idx)
target_mask = create_mask(target, pad_idx)

# Use masks to ignore padding in loss calculation
loss = criterion(predictions, target) * target_mask
loss = loss.sum() / target_mask.sum()
""")

# STEP 10: Save configuration for model
config = {
    "tokenization": "character_level",
    "framework": "pytorch",
    "urdu_vocab_size": urdu_char_vocab['vocab_size'],
    "roman_vocab_size": roman_char_vocab['vocab_size'],
    "max_sequence_length": max_length,
    "special_tokens": {
        "pad": "<pad>",
        "unk": "<unk>",
        "sos": "<sos>",
        "eos": "<eos>"
    },
    "data_folder": char_folder
}

with open(f"{char_folder}/pytorch_config.json", "w", encoding="utf-8") as f:
    json.dump(config, f, ensure_ascii=False, indent=2)

print(f"\nConfiguration saved to: {char_folder}/pytorch_config.json")

Dataset sizes:
Train: 10501, Validation: 5251, Test: 5251

Building character vocabularies...
Building Urdu character vocabulary...
Processing 10501 texts for vocabulary...
  Processed 1000 texts...
  Processed 2000 texts...
  Processed 3000 texts...
  Processed 4000 texts...
  Processed 5000 texts...
  Processed 6000 texts...
  Processed 7000 texts...
  Processed 8000 texts...
  Processed 9000 texts...
  Processed 10000 texts...
Found 47 unique characters before filtering
Building Roman Urdu character vocabulary...
Processing 10501 texts for vocabulary...
  Processed 1000 texts...
  Processed 2000 texts...
  Processed 3000 texts...
  Processed 4000 texts...
  Processed 5000 texts...
  Processed 6000 texts...
  Processed 7000 texts...
  Processed 8000 texts...
  Processed 9000 texts...
  Processed 10000 texts...
Found 30 unique characters before filtering

Urdu character vocabulary size: 51
Roman Urdu character vocabulary size: 34

Converting texts to character sequences...
Converting 

# =============================================================================
# ASSIGNMENT PROGRESS: PREPROCESSING COMPLETED
# =============================================================================

# ✅ TASK 1: PREPROCESSING - COMPLETED
# -----------------------------------------------------------------------------

## 1. DATA CLEANING:
 ✓ Normalized Urdu characters (removed diacritics, standardized Yeh/Alef forms)
 ✓ Cleaned Roman Urdu text (lowercased, removed punctuation, normalized spacing)
 ✓ Extracted Urdu → Roman Urdu pairs from dataset structure

## 2. DATA SPLITTING (50%/25%/25%):
 ✓ Split into Train (50%), Validation (25%), Test (25%) as required
 ✓ Used random_state=42 for reproducible splits
 ✓ Ensured no data leakage between sets

## 3. TOKENIZATION & VOCABULARY BUILDING:
 ✓ Created separate tokenizers for Urdu (source) and Roman Urdu (target)
 ✓ Built vocabularies using ONLY training data (prevents data leakage)
 ✓ Added special tokens: <unk>, <pad>, <sos>, <eos>
 ✓ Converted text to numerical sequences for neural network input
 ✓ Padded sequences to fixed length (50 tokens) for batch processing

## 4. DATA PREPARATION:
 ✓ Saved processed data as PyTorch tensors
 ✓ Saved vocabularies for future use
 ✓ All preprocessing done in pure PyTorch as required

## =============================================================================
## NEXT STEPS: MODEL ARCHITECTURE (BiLSTM Encoder + LSTM Decoder)
## =============================================================================
 - Build 2-layer BiLSTM encoder
 - Build 4-layer LSTM decoder  
 - Set up training loop with cross-entropy loss
 - Implement BLEU score evaluation
## =============================================================================

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
import math
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction
import nltk
import time
import random
import Levenshtein

# Download NLTK data
try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

# Set random seeds
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
print(f"PyTorch version: {torch.__version__}")

# =============================================================================
# PYTORCH DATASET CLASS
# =============================================================================
class UrduRomanDataset(Dataset):
    """PyTorch Dataset for Urdu to Roman Urdu translation"""

    def __init__(self, urdu_sequences, roman_sequences):
        self.urdu_sequences = torch.tensor(urdu_sequences, dtype=torch.long)
        self.roman_sequences = torch.tensor(roman_sequences, dtype=torch.long)

    def __len__(self):
        return len(self.urdu_sequences)

    def __getitem__(self, idx):
        urdu_seq = self.urdu_sequences[idx]
        roman_seq = self.roman_sequences[idx]

        # Teacher forcing: decoder input is roman_seq[:-1], target is roman_seq[1:]
        decoder_input = roman_seq[:-1]
        target = roman_seq[1:]

        return {
            'encoder_input': urdu_seq,
            'decoder_input': decoder_input,
            'target': target
        }

# =============================================================================
# BILSTM ENCODER (NO ATTENTION)
# =============================================================================
class BiLSTMEncoder(nn.Module):
    """BiLSTM Encoder - returns only final states"""

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super(BiLSTMEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout_rate)

        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            bidirectional=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        # Project bidirectional states to single direction
        self.h_projection = nn.Linear(hidden_dim * 2, hidden_dim)
        self.c_projection = nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        """
        Args:
            x: (batch, seq_len) - input sequences

        Returns:
            hidden: (batch, hidden_dim) - final hidden state
            cell: (batch, hidden_dim) - final cell state
        """
        embedded = self.dropout(self.embedding(x))

        # LSTM forward pass
        outputs, (hidden, cell) = self.lstm(embedded)

        # hidden and cell shape: (num_layers * 2, batch, hidden_dim)
        hidden = hidden.view(self.num_layers, 2, -1, self.hidden_dim)
        cell = cell.view(self.num_layers, 2, -1, self.hidden_dim)

        # Concatenate forward and backward for last layer
        hidden_last = torch.cat([hidden[-1, 0], hidden[-1, 1]], dim=1)
        cell_last = torch.cat([cell[-1, 0], cell[-1, 1]], dim=1)

        # Project to decoder dimension
        hidden_projected = torch.tanh(self.h_projection(hidden_last))
        cell_projected = torch.tanh(self.c_projection(cell_last))

        return hidden_projected, cell_projected

# =============================================================================
# LSTM DECODER (NO ATTENTION)
# =============================================================================
class LSTMDecoder(nn.Module):
    """Multi-layer LSTM Decoder without Attention"""

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, dropout_rate):
        super(LSTMDecoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        self.vocab_size = vocab_size

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.dropout = nn.Dropout(dropout_rate)

        # Simple LSTM decoder
        self.lstm = nn.LSTM(
            embedding_dim,
            hidden_dim,
            num_layers,
            batch_first=True,
            dropout=dropout_rate if num_layers > 1 else 0
        )

        # Output layer
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x, hidden, cell):
        """
        Args:
            x: (batch, seq_len) - decoder input
            hidden: (batch, hidden_dim) - initial hidden state
            cell: (batch, hidden_dim) - initial cell state

        Returns:
            predictions: (batch, seq_len, vocab_size) - output logits
        """
        # Expand hidden and cell to match num_layers
        hidden_expanded = hidden.unsqueeze(0).repeat(self.num_layers, 1, 1)
        cell_expanded = cell.unsqueeze(0).repeat(self.num_layers, 1, 1)

        # Embed input
        embedded = self.dropout(self.embedding(x))

        # LSTM forward
        output, (hidden_out, cell_out) = self.lstm(embedded, (hidden_expanded, cell_expanded))

        # Output layer (logits)
        predictions = self.fc(output)

        return predictions

# =============================================================================
# SEQ2SEQ MODEL (NO ATTENTION)
# =============================================================================
class Seq2Seq(nn.Module):
    """Complete Seq2Seq Model without Attention"""

    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, encoder_input, decoder_input):
        # Encode
        hidden, cell = self.encoder(encoder_input)

        # Decode
        outputs = self.decoder(decoder_input, hidden, cell)

        return outputs

# =============================================================================
# GREEDY DECODER (NO ATTENTION, NO BEAM SEARCH)
# =============================================================================
class GreedyDecoder:
    """Simple greedy decoding"""

    def __init__(self, model, roman_vocab, max_length=100):
        self.model = model
        self.roman_vocab = roman_vocab
        self.max_length = max_length
        self.eos_token = roman_vocab['char_to_idx']['<eos>']
        self.sos_token = roman_vocab['char_to_idx']['<sos>']
        self.pad_token = roman_vocab['char_to_idx']['<pad>']
        self.device = next(model.parameters()).device

    def decode(self, encoder_input):
        """Greedy decoding"""
        self.model.eval()
        with torch.no_grad():
            # Encode
            encoder_input = encoder_input.unsqueeze(0).to(self.device)
            hidden, cell = self.model.encoder(encoder_input)

            # Start with SOS token
            decoder_input = torch.full((1, 1), self.sos_token, dtype=torch.long).to(self.device)
            generated_sequence = [self.sos_token]

            # Expand hidden and cell for multi-layer decoder
            num_layers = self.model.decoder.num_layers
            hidden_expanded = hidden.unsqueeze(0).repeat(num_layers, 1, 1)
            cell_expanded = cell.unsqueeze(0).repeat(num_layers, 1, 1)

            # Greedy decoding
            for step in range(self.max_length - 1):
                # Embed current token
                embedded = self.model.decoder.dropout(
                    self.model.decoder.embedding(decoder_input)
                )

                # LSTM forward
                lstm_out, (hidden_expanded, cell_expanded) = self.model.decoder.lstm(
                    embedded, (hidden_expanded, cell_expanded)
                )

                # Get logits
                logits = self.model.decoder.fc(lstm_out[:, -1, :])

                # Get next token (greedy)
                next_token = torch.argmax(logits, dim=-1).item()

                # Check for EOS
                if next_token == self.eos_token:
                    break

                generated_sequence.append(next_token)

                # Prepare next decoder input
                decoder_input = torch.tensor([[next_token]], dtype=torch.long).to(self.device)

            # Remove SOS token
            return generated_sequence[1:]

# =============================================================================
# DATA PREPROCESSOR
# =============================================================================
class DataPreprocessor:
    """Handles sequence to text conversion"""

    def __init__(self):
        self.urdu_vocab = None
        self.roman_vocab = None

    def set_vocabularies(self, urdu_vocab, roman_vocab):
        self.urdu_vocab = urdu_vocab
        self.roman_vocab = roman_vocab

    def sequence_to_text(self, sequence, vocab_type='roman'):
        vocab = self.roman_vocab if vocab_type == 'roman' else self.urdu_vocab

        idx_to_char = vocab['idx_to_char']
        text = ''

        for idx in sequence:
            idx = int(idx)
            if idx in [vocab['char_to_idx']['<pad>'],
                      vocab['char_to_idx']['<sos>'],
                      vocab['char_to_idx']['<eos>']]:
                continue
            text += idx_to_char.get(idx, '<unk>')

        return text

# =============================================================================
# MASKED LOSS
# =============================================================================
class MaskedCrossEntropyLoss(nn.Module):
    """Cross Entropy Loss with padding mask"""

    def __init__(self, pad_idx=0):
        super(MaskedCrossEntropyLoss, self).__init__()
        self.pad_idx = pad_idx
        self.criterion = nn.CrossEntropyLoss(ignore_index=pad_idx, reduction='mean')

    def forward(self, predictions, targets):
        batch_size, seq_len, vocab_size = predictions.shape
        predictions = predictions.reshape(-1, vocab_size)
        targets = targets.reshape(-1)
        loss = self.criterion(predictions, targets)
        return loss

# =============================================================================
# TRAINER CLASS
# =============================================================================
class Trainer:
    """Training pipeline"""

    def __init__(self, model, train_loader, val_loader, test_loader,
                 urdu_vocab, roman_vocab, config):
        self.model = model.to(device)
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.test_loader = test_loader
        self.urdu_vocab = urdu_vocab
        self.roman_vocab = roman_vocab
        self.config = config

        self.criterion = MaskedCrossEntropyLoss(pad_idx=0)
        self.optimizer = optim.Adam(
            model.parameters(),
            lr=config['learning_rate'],
            betas=(0.9, 0.999)
        )
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer, mode='min', factor=0.5, patience=4
        )

        self.preprocessor = DataPreprocessor()
        self.preprocessor.set_vocabularies(urdu_vocab, roman_vocab)

        self.best_val_loss = float('inf')
        self.patience_counter = 0
        self.early_stop_patience = 8

    def calculate_accuracy(self, predictions, targets):
        """Calculate masked accuracy"""
        pred_tokens = predictions.argmax(dim=2)
        mask = (targets != 0).float()
        correct = (pred_tokens == targets).float() * mask
        accuracy = correct.sum() / (mask.sum() + 1e-8)
        return accuracy.item()

    def calculate_cer(self, predicted, target):
        """Calculate Character Error Rate"""
        if len(target) == 0:
            return 1.0 if len(predicted) > 0 else 0.0
        distance = Levenshtein.distance(predicted, target)
        return distance / len(target)

    def calculate_edit_distance(self, predicted, target):
        """Calculate Levenshtein edit distance"""
        return Levenshtein.distance(predicted, target)

    def train_epoch(self):
        self.model.train()
        total_loss = 0
        total_accuracy = 0

        for batch in self.train_loader:
            encoder_input = batch['encoder_input'].to(device)
            decoder_input = batch['decoder_input'].to(device)
            target = batch['target'].to(device)

            self.optimizer.zero_grad()

            # Forward pass
            outputs = self.model(encoder_input, decoder_input)

            # Calculate loss
            loss = self.criterion(outputs, target)

            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()

            # Calculate accuracy
            accuracy = self.calculate_accuracy(outputs, target)

            total_loss += loss.item()
            total_accuracy += accuracy

        avg_loss = total_loss / len(self.train_loader)
        avg_accuracy = total_accuracy / len(self.train_loader)

        return avg_loss, avg_accuracy

    def validate(self):
        self.model.eval()
        total_loss = 0
        total_accuracy = 0

        with torch.no_grad():
            for batch in self.val_loader:
                encoder_input = batch['encoder_input'].to(device)
                decoder_input = batch['decoder_input'].to(device)
                target = batch['target'].to(device)

                outputs = self.model(encoder_input, decoder_input)
                loss = self.criterion(outputs, target)
                accuracy = self.calculate_accuracy(outputs, target)

                total_loss += loss.item()
                total_accuracy += accuracy

        avg_loss = total_loss / len(self.val_loader)
        avg_accuracy = total_accuracy / len(self.val_loader)

        return avg_loss, avg_accuracy

    def train(self, num_epochs):
        print(f"\nStarting training for {num_epochs} epochs...")

        for epoch in range(num_epochs):
            start_time = time.time()

            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc = self.validate()

            epoch_time = time.time() - start_time

            print(f"Epoch {epoch+1}/{num_epochs} ({epoch_time:.2f}s)")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
            print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

            # Learning rate scheduling
            self.scheduler.step(val_loss)

            # Early stopping
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                torch.save(self.model.state_dict(), f'best_model_{self.config["name"]}.pt')
                print(f"  Saved best model!")
            else:
                self.patience_counter += 1
                if self.patience_counter >= self.early_stop_patience:
                    print(f"\nEarly stopping at epoch {epoch+1}")
                    break

    def evaluate(self):
        """Evaluate on test set"""
        print("\n" + "="*80)
        print(f"EVALUATION ON TEST SET: {self.config['name']}")
        print("="*80)

        # Load best model
        self.model.load_state_dict(torch.load(f'best_model_{self.config["name"]}.pt'))

        # Setup greedy decoder
        decoder = GreedyDecoder(self.model, self.roman_vocab)

        # Evaluate on 10 samples
        test_data = []
        for batch in self.test_loader:
            for i in range(len(batch['encoder_input'])):
                test_data.append({
                    'encoder_input': batch['encoder_input'][i],
                    'target': batch['target'][i]
                })
                if len(test_data) >= 10:
                    break
            if len(test_data) >= 10:
                break

        references = []
        hypotheses = []
        cer_scores = []
        edit_distances = []

        print("\nQUALITATIVE EXAMPLES (Predictions vs Ground Truth):")
        print("-" * 100)

        for i, sample in enumerate(test_data):
            urdu_seq = sample['encoder_input'].cpu().numpy()
            target_seq = sample['target'].cpu().numpy()

            urdu_text = self.preprocessor.sequence_to_text(urdu_seq, 'urdu')
            target_roman = self.preprocessor.sequence_to_text(target_seq, 'roman')

            # Greedy decoding
            predicted_seq = decoder.decode(sample['encoder_input'])
            predicted_roman = self.preprocessor.sequence_to_text(predicted_seq, 'roman')

            # Calculate metrics
            cer = self.calculate_cer(predicted_roman, target_roman)
            edit_dist = self.calculate_edit_distance(predicted_roman, target_roman)

            references.append([target_roman.split()])
            hypotheses.append(predicted_roman.split())
            cer_scores.append(cer)
            edit_distances.append(edit_dist)

            print(f"\nSample {i+1}:")
            print(f"  Urdu:      {urdu_text}")
            print(f"  Target:    {target_roman}")
            print(f"  Predicted: {predicted_roman}")
            print(f"  CER:       {cer:.4f}")
            print(f"  Edit Dist: {edit_dist}")
            print("-" * 100)

        # Calculate BLEU score
        smoothie = SmoothingFunction().method4
        bleu_score = corpus_bleu(references, hypotheses, smoothing_function=smoothie)

        # Average CER and Edit Distance
        avg_cer = np.mean(cer_scores)
        avg_edit_dist = np.mean(edit_distances)

        print(f"\nAGGREGATE METRICS:")
        print(f"  BLEU Score:           {bleu_score:.4f}")
        print(f"  Average CER:          {avg_cer:.4f}")
        print(f"  Average Edit Distance: {avg_edit_dist:.2f}")

        # Test set metrics
        test_loss, test_acc = self.validate()
        perplexity = math.exp(test_loss) if test_loss < 100 else float('inf')

        print(f"\nMODEL METRICS:")
        print(f"  Test Loss:     {test_loss:.4f}")
        print(f"  Test Accuracy: {test_acc:.4f}")
        print(f"  Perplexity:    {perplexity:.4f}")

        return {
            'loss': test_loss,
            'accuracy': test_acc,
            'bleu_score': bleu_score,
            'perplexity': perplexity,
            'avg_cer': avg_cer,
            'avg_edit_distance': avg_edit_dist
        }

# =============================================================================
# MAIN EXECUTION
# =============================================================================
def load_char_vocab(filepath):
    """Load character vocabulary"""
    with open(filepath, 'r', encoding='utf-8') as f:
        vocab_data = json.load(f)
    vocab_data['idx_to_char'] = {int(k): v for k, v in vocab_data['idx_to_char'].items()}
    return vocab_data

def main():
    print("="*80)
    print("Urdu to Roman Urdu Translation WITHOUT Attention (Baseline)")
    print("="*80)

    # Load preprocessed data
    folder = "Processed_Char_level_PyTorch"

    train_urdu = np.load(f"{folder}/train_urdu.npy")
    train_roman = np.load(f"{folder}/train_roman.npy")
    val_urdu = np.load(f"{folder}/val_urdu.npy")
    val_roman = np.load(f"{folder}/val_roman.npy")
    test_urdu = np.load(f"{folder}/test_urdu.npy")
    test_roman = np.load(f"{folder}/test_roman.npy")

    urdu_vocab = load_char_vocab(f"{folder}/urdu_char_vocab.json")
    roman_vocab = load_char_vocab(f"{folder}/roman_char_vocab.json")

    print(f"\nDataset Statistics:")
    print(f"  Train: {len(train_urdu)} samples")
    print(f"  Validation: {len(val_urdu)} samples")
    print(f"  Test: {len(test_urdu)} samples")
    print(f"  Urdu vocab size: {urdu_vocab['vocab_size']}")
    print(f"  Roman vocab size: {roman_vocab['vocab_size']}")

    # Create datasets
    train_dataset = UrduRomanDataset(train_urdu, train_roman)
    val_dataset = UrduRomanDataset(val_urdu, val_roman)
    test_dataset = UrduRomanDataset(test_urdu, test_roman)

    # Define THREE experiments - WITHOUT ATTENTION
    experiments = [
        {
            'name': 'Exp1-BiLSTM2-LSTM4-NoAttention',
            'embedding_dim': 256,
            'hidden_dim': 256,
            'encoder_layers': 2,
            'decoder_layers': 4,
            'dropout': 0.2,
            'learning_rate': 0.002,
            'batch_size': 64,
            'epochs': 20
        },
        {
            'name': 'Exp2-BiLSTM3-LSTM3-NoAttention',
            'embedding_dim': 128,
            'hidden_dim': 256,
            'encoder_layers': 3,
            'decoder_layers': 3,
            'dropout': 0.3,
            'learning_rate': 0.001,
            'batch_size': 64,
            'epochs': 20
        },
        {
            'name': 'Exp3-BiLSTM2-LSTM2-Large-NoAttention',
            'embedding_dim': 256,
            'hidden_dim': 512,
            'encoder_layers': 2,
            'decoder_layers': 2,
            'dropout': 0.1,
            'learning_rate': 0.0005,
            'batch_size': 32,
            'epochs': 20
        }
    ]

    results = {}

    print("\n" + "="*80)
    print("STARTING EXPERIMENTS (WITHOUT ATTENTION)")
    print("="*80)

    for exp_num, config in enumerate(experiments, 1):
        print(f"\n{'='*80}")
        print(f"EXPERIMENT {exp_num}/3: {config['name']}")
        print(f"{'='*80}")
        print(f"Configuration:")
        print(f"  Embedding Dim: {config['embedding_dim']}")
        print(f"  Hidden Dim: {config['hidden_dim']}")
        print(f"  Encoder Layers: {config['encoder_layers']} (BiLSTM)")
        print(f"  Decoder Layers: {config['decoder_layers']} (LSTM - No Attention)")
        print(f"  Dropout: {config['dropout']}")
        print(f"  Learning Rate: {config['learning_rate']}")
        print(f"  Batch Size: {config['batch_size']}")
        print(f"  Max Epochs: {config['epochs']}")

        # Create data loaders
        train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=config['batch_size'], shuffle=False)

        # Build model WITHOUT attention
        encoder = BiLSTMEncoder(
            urdu_vocab['vocab_size'],
            config['embedding_dim'],
            config['hidden_dim'],
            config['encoder_layers'],
            config['dropout']
        )

        decoder = LSTMDecoder(
            roman_vocab['vocab_size'],
            config['embedding_dim'],
            config['hidden_dim'],
            config['decoder_layers'],
            config['dropout']
        )

        model = Seq2Seq(encoder, decoder)

        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"\nModel Statistics:")
        print(f"  Total Parameters: {total_params:,}")
        print(f"  Trainable Parameters: {trainable_params:,}")

        # Train
        trainer = Trainer(model, train_loader, val_loader, test_loader,
                         urdu_vocab, roman_vocab, config)
        trainer.train(config['epochs'])

        # Evaluate
        test_results = trainer.evaluate()
        results[config['name']] = {
            'config': config,
            'results': test_results
        }

        print(f"\n{config['name']} Completed!")

    # Final comprehensive results
    print("\n" + "="*80)
    print("FINAL COMPARISON (WITHOUT ATTENTION)")
    print("="*80)

    print(f"\n{'Experiment':<45} {'BLEU':<10} {'CER':<10} {'Edit Dist':<12} {'Accuracy':<10} {'Perplexity':<12}")
    print("-" * 105)

    for name, data in results.items():
        res = data['results']
        print(f"{name:<45} {res['bleu_score']:<10.4f} {res['avg_cer']:<10.4f} {res['avg_edit_distance']:<12.2f} {res['accuracy']:<10.4f} {res['perplexity']:<12.4f}")

    print("\n" + "="*80)
    print("DETAILED RESULTS")
    print("="*80)

    for name, data in results.items():
        config = data['config']
        res = data['results']

        print(f"\n{name}:")
        print(f"  Configuration:")
        print(f"    Embedding: {config['embedding_dim']}, Hidden: {config['hidden_dim']}")
        print(f"    Encoder: BiLSTM-{config['encoder_layers']}, Decoder: LSTM-{config['decoder_layers']} (No Attention)")
        print(f"    Dropout: {config['dropout']}, LR: {config['learning_rate']}, Batch: {config['batch_size']}")
        print(f"  Results:")
        print(f"    BLEU Score: {res['bleu_score']:.4f}")
        print(f"    Character Error Rate (CER): {res['avg_cer']:.4f}")
        print(f"    Edit Distance (Levenshtein): {res['avg_edit_distance']:.2f}")
        print(f"    Accuracy: {res['accuracy']:.4f}")
        print(f"    Perplexity: {res['perplexity']:.4f}")
        print(f"    Loss: {res['loss']:.4f}")

    # Find best model
    if results:
        best_bleu_model = max(results.items(), key=lambda x: x[1]['results']['bleu_score'])
        best_cer_model = min(results.items(), key=lambda x: x[1]['results']['avg_cer'])

        print("\n" + "="*80)
        print("BEST MODELS (WITHOUT ATTENTION)")
        print("="*80)
        print(f"\nBest by BLEU Score:")
        print(f"  Model: {best_bleu_model[0]}")
        print(f"  BLEU: {best_bleu_model[1]['results']['bleu_score']:.4f}")

        print(f"\nBest by Character Error Rate (CER):")
        print(f"  Model: {best_cer_model[0]}")
        print(f"  CER: {best_cer_model[1]['results']['avg_cer']:.4f}")

        print("\n" + "="*80)
        print("BASELINE EXPERIMENTS COMPLETED!")
        print("="*80)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Using device: cuda
PyTorch version: 2.8.0+cu126
Urdu to Roman Urdu Translation WITHOUT Attention (Baseline)

Dataset Statistics:
  Train: 10501 samples
  Validation: 5251 samples
  Test: 5251 samples
  Urdu vocab size: 51
  Roman vocab size: 34

STARTING EXPERIMENTS (WITHOUT ATTENTION)

EXPERIMENT 1/3: Exp1-BiLSTM2-LSTM4-NoAttention
Configuration:
  Embedding Dim: 256
  Hidden Dim: 256
  Encoder Layers: 2 (BiLSTM)
  Decoder Layers: 4 (LSTM - No Attention)
  Dropout: 0.2
  Learning Rate: 0.002
  Batch Size: 64
  Max Epochs: 20

Model Statistics:
  Total Parameters: 5,028,130
  Trainable Parameters: 5,028,130

Starting training for 20 epochs...
